In [1]:
%load_ext py5
%gui osx

In [2]:
import pandas as pd
import numpy as np
import scanpy as sc

In [5]:
print('DOWNLOADING AND EXTRACTING EXAMPLE DATA')
! mkdir -p ../../data
! wget https://www.dropbox.com/s/kmryxbttn7e0wh3/pbmc3k_20210420.h5ad -O ../../data/pbmc3k_20210420.h5ad
! ls ../../data

DOWNLOADING AND EXTRACTING EXAMPLE DATA
--2021-07-06 12:24:34--  https://www.dropbox.com/s/kmryxbttn7e0wh3/pbmc3k_20210420.h5ad
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kmryxbttn7e0wh3/pbmc3k_20210420.h5ad [following]
--2021-07-06 12:24:34--  https://www.dropbox.com/s/raw/kmryxbttn7e0wh3/pbmc3k_20210420.h5ad
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca7a85c47983de2c5d0ee899717.dl.dropboxusercontent.com/cd/0/inline/BRx9q2nHvWMuabozSrvjUvPTaVbO5nndt40KOx8TfFgAM8EWTaQGM5QIwRNHq8zW3jbdCLpxjm9kfCfv-qOeMdvIpvRb7Y9YTgI0-17FJgNL83pQFrcrlDOhXVs29Ac2qZwLVvPRUuJbr4z8JdCFRHAg/file# [following]
--2021-07-06 12:24:34--  https://uca7a85c47983de2c5d0ee899717.dl.dropboxusercontent.com/cd/0/inline/BRx9q2nHvWMuabozSrvjUvPTaVbO5nndt40KOx8TfFgAM8EWTaQGM5

In [3]:
print("LOADING DATA...")
data = sc.read('../../data/pbmc3k_20210420.h5ad')
data

LOADING DATA...


AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
data.raw.X

<2638x13714 sparse matrix of type '<class 'numpy.float32'>'
	with 2238732 stored elements in Compressed Sparse Row format>

In [5]:
## Convert the sparse data in the AnnData.raw.X attribute from csr_matrix to csc_matrix for compatibility with sciviewer.
import scipy.sparse as sp
z = sp.csc.csc_matrix(data.raw.X)
raw = sc.AnnData(z, obs=data.obs, var=data.raw.var)
data.raw = raw

In [6]:
data.raw.X

<2638x13714 sparse matrix of type '<class 'numpy.float32'>'
	with 2238732 stored elements in Compressed Sparse Column format>

In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../../sciviewer')
from sciviewer import SCIViewer

## Remove the relative import . module imports in sciviewer.py and gui.py for this to work

## This try / except block doesn't seem to close the sketch
## on crashing for some reason
try:
    # use_raw=True tells sciviewer to compute correlations on the raw data containing all genes in sparse format
    # rather than the main data which only includes high-variance genes 
    svobj = SCIViewer(data, use_raw=True)                      
    svobj.explore_data()
except:
    svobj.renderer.exit_sketch()

0.18015003204345703 seconds to select and project cells
fo
Selected 1607 cells
Calculating correlations...
1.4028277397155762 seconds to calculate correlations. Sparsity:  True
Selected gene RPL32
Min/max expression level for gene RPL32 0.0 5.7993245
Selected gene NKG7
Min/max expression level for gene NKG7 0.0 5.948639
EXPORTING DATA...
BYE


## The class attributes below get populated in real time and thus can be accessed when the interactive viewer is running

In [21]:
## This attribute get updated in real time whenever cells are selected

svobj.selected_cells.head()

,index,cell_name,projection
0,1,AAACATTGAGCTAC-1,0.924999
1,10,AAACTTGAAAAACG-1,0.928325
2,18,AAAGGCCTGTCTAG-1,0.863444
3,19,AAAGTTTGATCACG-1,0.114750
4,20,AAAGTTTGGGGTGA-1,0.628166


In [22]:
## This gets updated in real time when cells are selected in differential expression mode
svobj.results_proj_correlation.head()

,R,P
AL627309.1,NaN,NaN
AP006222.2,NaN,NaN
RP11-206L10.2,0.018435,0.734832
RP11-206L10.9,NaN,NaN
LINC00115,-0.041971,0.440474


In [23]:
## This gets updated in real time when cells are selected in directional correlation mode
svobj.results_diffexpr.head()

,T,P
AL627309.1,-2.985351,0.002859
AP006222.2,-1.724507,0.084735
RP11-206L10.2,1.375107,0.178075
RP11-206L10.9,-1.728097,0.084089
LINC00115,-4.224138,0.000025


In [24]:
svobj.results_proj_correlation.head()

,R,P
AL627309.1,NaN,NaN
AP006222.2,NaN,NaN
RP11-206L10.2,0.018435,0.734832
RP11-206L10.9,NaN,NaN
LINC00115,-0.041971,0.440474


## The class attributes below only get populated when the export and close button is pushed

In [25]:
svobj.significant_genes.head()

,R,P
IGJ,0.346487,5.031513e-11
S100A10,0.286611,7.505241e-08
AIM2,0.283531,1.047175e-07
COTL1,0.279396,1.627554e-07
VIM,0.268011,5.283376e-07


In [26]:
svobj.significant_genes.tail()

,R,P
SNX29P2,-0.292041,4.130506e-08
CD79B,-0.294465,3.151355e-08
CD72,-0.306601,7.822962e-09
CD74,-0.306615,7.809669e-09
TCL1A,-0.436444,3.034777e-17


In [27]:
svobj.selected_gene_cell_data.head()

,index,proj,exp
0,AAACATTGAGCTAC-1,0.924999,4.309784
1,AAACTTGAAAAACG-1,0.928325,4.532358
2,AAAGGCCTGTCTAG-1,0.863444,3.301051
3,AAAGTTTGATCACG-1,0.114750,5.016235
4,AAAGTTTGGGGTGA-1,0.628166,4.458291
